In [19]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.nn.utils.rnn import pad_sequence
import numpy as np


df = pd.read_csv("netflix_reviews.csv")  # 파일 불러오기
df = df.iloc[:,0:5]

# 전처리 함수
import re
def preprocess_text(text):
    if isinstance(text, float):
        return ""
    text = text.lower()  # 대문자를 소문자로
    text = re.sub(r'[^\w\s]', '', text)  # 구두점 제거
    text = re.sub(r'\d+', '', text)  # 숫자 제거
    text = text.strip()  # 띄어쓰기 제외하고 빈 칸 제거
    return text


df['reviewId'] = df['reviewId'].apply(preprocess_text)
df['userName'] = df['userName'].apply(preprocess_text)
df['content'] = df['content'].apply(preprocess_text)


#
reviews = df['content'].tolist()  # 'content'를 리스트로 변환
ratings = df['score'].tolist()    # 'score'를 리스트로 변환



# 라벨을 정수형으로 변환 (필수적인 과정)
label_encoder = LabelEncoder()
ratings = label_encoder.fit_transform(ratings)  # 평점 정수형으로 변환

# 데이터셋 클래스 정의
class ReviewDataset(Dataset):
    def __init__(self, reviews, ratings, text_pipeline, label_pipeline):
        self.reviews = reviews
        self.ratings = ratings
        self.text_pipeline = text_pipeline
        self.label_pipeline = label_pipeline

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = self.text_pipeline(self.reviews[idx])
        rating = self.label_pipeline(self.ratings[idx])
        return torch.tensor(review), torch.tensor(rating)

# 토크나이저 정의 (기본 영어 토크나이저)
tokenizer = get_tokenizer('basic_english')

# 어휘 사전 생성 함수
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

# 어휘 사전 생성
vocab = build_vocab_from_iterator(yield_tokens(reviews))


# 텍스트 파이프라인 정의 (어휘 사전에 있는 단어만 처리)
def text_pipeline(text):
    return [vocab[token] for token in tokenizer(text)]

# 평점 그대로 사용
def label_pipeline(label):
    return label  # 이미 숫자형이므로 변환 생략

# 데이터를 학습용(train)과 테스트용(test)으로 분리
train_reviews, test_reviews, train_ratings, test_ratings = train_test_split(reviews, ratings, test_size=0.2, random_state=42)

# 데이터셋 정의
train_dataset = ReviewDataset(train_reviews, train_ratings, text_pipeline, label_pipeline)
test_dataset = ReviewDataset(test_reviews, test_ratings, text_pipeline, label_pipeline)

# 패딩을 적용하는 함수 정의

def collate_fn(batch):
    reviews, ratings = zip(*batch)
    reviews = pad_sequence([torch.tensor(r, dtype=torch.long) for r in reviews], batch_first=True)  # 정수형 텐서로 변환
    ratings = torch.tensor(ratings, dtype=torch.long)  # 평점도 정수형으로 변환
    return reviews, ratings
# 데이터 로더 정의
BATCH_SIZE = 64

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)  # Embedding으로 변경
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        output, (hidden, cell) = self.lstm(embedded)
        return self.fc(hidden[-1])

# 하이퍼파라미터 정의
VOCAB_SIZE = len(vocab)
EMBED_DIM = 64
HIDDEN_DIM = 128
OUTPUT_DIM = len(set(ratings))  # 예측할 점수 개수 (평점이 정수형)

# 모델 초기화
model = LSTMModel(VOCAB_SIZE, EMBED_DIM, HIDDEN_DIM, OUTPUT_DIM)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)  # SGD 에서 Adam으로 변경 lr : 0.01 - > 0.001 / Accuracy: 63% -> 61.59% 다시 0.01

# 모델을 CUDA로 이동 (가능한 경우)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 모델 학습 함수 정의
def train_model(model, train_dataloader, criterion, optimizer, num_epochs=10):
    model.train()  # 학습 모드로 설정
    for epoch in range(num_epochs):
        total_loss = 0  # 에포크마다 손실을 추적
        for i, (reviews, ratings) in enumerate(train_dataloader):
            reviews, ratings = reviews.to(device), ratings.to(device)  # 데이터를 GPU로 이동
            
            optimizer.zero_grad()
            outputs = model(reviews)  # 모델에 입력하여 예측값 계산
            loss = criterion(outputs, ratings)  # 손실 계산
            loss.backward()  # 역전파
            optimizer.step()  # 가중치 업데이트

            total_loss += loss.item()

            # 배치마다 손실 출력
            if (i + 1) % 10 == 0:
                print(f'Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(train_dataloader)}, Loss: {loss.item():.4f}')
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {total_loss/len(train_dataloader):.4f}')
    
    print("Finished Training")

# 모델 학습 실행
train_model(model, train_dataloader, criterion, optimizer, num_epochs=10)

# 모델 평가

correct = 0
total = 0
with torch.no_grad():  # 평가 시에는 기울기 계산을 하지 않음
    for reviews, ratings in test_dataloader:
        reviews, ratings = reviews.to(device), ratings.to(device)
        outputs = model(reviews)
        _, predicted = torch.max(outputs, 1)
        total += ratings.size(0)
        correct += (predicted == ratings).sum().item()

print(f'Accuracy: {100 * correct / total}%')


117134lines [00:01, 85451.69lines/s] 
C:\Users\1\AppData\Local\Temp\ipykernel_18016\4051435007.py:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  reviews = pad_sequence([torch.tensor(r, dtype=torch.long) for r in reviews], batch_first=True)  # 정수형 텐서로 변환


Epoch 1/10, Batch 10/1465, Loss: 1.4973
Epoch 1/10, Batch 20/1465, Loss: 1.5300
Epoch 1/10, Batch 30/1465, Loss: 1.3624
Epoch 1/10, Batch 40/1465, Loss: 1.5449
Epoch 1/10, Batch 50/1465, Loss: 1.4053
Epoch 1/10, Batch 60/1465, Loss: 1.4345
Epoch 1/10, Batch 70/1465, Loss: 1.4330
Epoch 1/10, Batch 80/1465, Loss: 1.5878
Epoch 1/10, Batch 90/1465, Loss: 1.4180
Epoch 1/10, Batch 100/1465, Loss: 1.4489
Epoch 1/10, Batch 110/1465, Loss: 1.5973
Epoch 1/10, Batch 120/1465, Loss: 1.3844
Epoch 1/10, Batch 130/1465, Loss: 1.3032
Epoch 1/10, Batch 140/1465, Loss: 1.4516
Epoch 1/10, Batch 150/1465, Loss: 1.3552
Epoch 1/10, Batch 160/1465, Loss: 1.5032
Epoch 1/10, Batch 170/1465, Loss: 1.5434
Epoch 1/10, Batch 180/1465, Loss: 1.5632
Epoch 1/10, Batch 190/1465, Loss: 1.4780
Epoch 1/10, Batch 200/1465, Loss: 1.4367
Epoch 1/10, Batch 210/1465, Loss: 1.4150
Epoch 1/10, Batch 220/1465, Loss: 1.4449
Epoch 1/10, Batch 230/1465, Loss: 1.3958
Epoch 1/10, Batch 240/1465, Loss: 1.4321
Epoch 1/10, Batch 250/146

In [21]:
# 예측 함수
def predict_review(model, review, vocab, tokenizer, device):
    # 리뷰를 텐서로 변환
    tokens = [vocab[token] for token in tokenizer(review)]
    review_tensor = torch.tensor(tokens).unsqueeze(0)  # (1, seq_length) 형태로 만듦
    
    # 텐서를 GPU로 이동
    review_tensor = review_tensor.to(device)
    
    # 모델에 입력하여 예측값 계산
    model.eval()  # 평가 모드로 변경
    with torch.no_grad():  # 평가 시에는 기울기 계산을 하지 않음
        output = model(review_tensor)
        _, predicted = torch.max(output, 1)
    
    return predicted.item()  # 예측된 평점 반환
# 새로운 리뷰에 대한 예측
new_review = "This app is great but has some bugs."
predicted_score = predict_review(model, new_review, vocab, tokenizer,  device)
print(f'Predicted Score: {predicted_score}')

Predicted Score: 4
